In [99]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [100]:
n = lk('../../lookup_tables/nfet_03v3.mat')
p = lk('../../lookup_tables/pfet_03v3.mat')

#### Specifications

In [101]:
GBW=100e6; CL=1e-12

#### Design choices

In [102]:
gm_id0 = 12
gm_id1 = 12
gm_id2 = 12
l0 = 0.5
l1 = 0.5
l2 = 0.5

#### Sizing and benchmarking

In [103]:
# calculate gm of differential pair
gm1 = 2*np.pi*GBW*CL

# size all transistors
id1 = gm1/gm_id1; id2=id1; id0=2*id1
jd1 = n.lookup('ID_W', GM_ID=gm_id1, L=l1)
w1 = id1/jd1
jd2 = p.lookup('ID_W', GM_ID=gm_id2, L=l2)
w2 = id1/jd2
w0 = 2*w1

# estimate mirror pole
cgg2 = w2*p.lookup('CGG_W', GM_ID=gm_id2, L=l2)
cdd2 = w2*p.lookup('CDD_W', GM_ID=gm_id2, L=l2)
cdd1 = w2*n.lookup('CDD_W', GM_ID=gm_id1, L=l1)
gm2 = gm1
fp2 = gm2/(2*cgg2+cdd1+cdd2)/(2*np.pi)

# estimate phase margin (mirror pole, LHP zero, RHP zero)
phip2 = -np.arctan(GBW/fp2)*180/np.pi
fz2 = 2*fp2
phiz2 = +np.arctan(GBW/fz2)*180/np.pi
cgd1 = w1*n.lookup('CGD_W', GM_ID=gm_id1, L=l1)
fz3 = gm1/cgd1/(2*np.pi)
phiz3 = -np.arctan(GBW/fz3)*180/np.pi
PM = 90 +phip2 +phiz2 +phiz3 

df = pd.DataFrame( [id1/1e-6, gm1/1e-3, fp2, PM], \
                   ['id1 (uA)', 'gm1 (mS)', 'fp2 (Hz)', 'PM (deg)'], columns=['Value']); df.round(2)

,Value
id1 (uA),5.236000e+01
gm1 (mS),6.300000e-01
fp2 (Hz),2.814258e+08
PM (deg),8.036000e+01


In [104]:
# finger the devices
wfing = 5
nf0 = 1+np.floor_divide(w0, wfing)
nf1 = 1+np.floor_divide(w1, wfing)
nf2 = 1+np.floor_divide(w2, wfing)
df = pd.DataFrame( [(id0*1e6, id1*1e6, id2*1e6), (w0, w1, w2), (l0, l1, l2), (nf0, nf1, nf2)], \
                   ['ID (uA)', 'w (um)', 'l (um)', 'nf'], columns=['M0', 'M1', 'M2']); df.round(2)

,M0,M1,M2
ID (uA),104.72,52.36,52.36
w (um),38.50,19.25,66.52
l (um),0.50,0.50,0.50
nf,8.00,4.00,14.00


#### Write spice include file

In [105]:
with open('sizing_ota-5t_gf.spice', 'w') as file:
    file.write(".param ibn = " + "{:.2e}".format(id0) + '\n')
    file.write(".param W0 = " + "{:.2e}".format(w0*1e-6) + '\n')
    file.write(".param W1 = " + "{:.2e}".format(w1*1e-6) + '\n')
    file.write(".param W2 = " + "{:.2e}".format(w2*1e-6) + '\n')
    file.write(".param L0 = " + "{:.2e}".format(l0*1e-6) + '\n')
    file.write(".param L1 = " + "{:.2e}".format(l1*1e-6) + '\n')
    file.write(".param L2 = " + "{:.2e}".format(l2*1e-6) + '\n')
    file.write(".param nf0 = " + "{}".format(nf0) + '\n')
    file.write(".param nf1 = " + "{}".format(nf1) + '\n')
    file.write(".param nf2 = " + "{}".format(nf2) + '\n')